In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

# Importing Data

In [2]:
#Function to import the data from the dir you are in
def import_from_dir(current_dir):
    files_lst = os.listdir(current_dir)
    df = pd.DataFrame()
    for file in files_lst:
        if file.endswith('.csv'):
            #attaching an empty dataframe to the df that wer read
            data = pd.read_csv(current_dir + file)
            df = pd.concat([df, data])
            df = df.drop_duplicates()
    return df

In [13]:
current_dir = './'
fifa_df=import_from_dir(current_dir).set_index('ID', drop=True)
fifa_df

,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,Club Logo,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender
ID,,,,,,,,,,,,,,,,,,,,,
2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,...,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,...,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,...,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,...,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,...,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259088,A. Medioub,22,64,France,CD Tondela,66,CB,CB,https://cdn.sofifa.com/players/259/088/20_120.png,https://cdn.sofifa.com/teams/112516/light_60.png,...,55+2,55+2,54+2,57+2,64+2,64+2,64+2,57+2,12+2,Male
259090,W. Rickard,18,56,Wales,Burnley,58,CAM,CAM CM,https://cdn.sofifa.com/players/259/090/20_120.png,https://cdn.sofifa.com/teams/1796/light_60.png,...,52+2,52+2,53+2,52+2,49+2,49+2,49+2,52+2,17+2,Male
259091,C. Barrett,18,56,England,Burnley,56,RB,RB,https://cdn.sofifa.com/players/259/091/20_120.png,https://cdn.sofifa.com/teams/1796/light_60.png,...,50+2,50+2,54+2,54+2,54+2,54+2,54+2,54+2,14+2,Male


In [11]:
fifa_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17125 entries, 0 to 17124
Columns: 107 entries, ID to Gender
dtypes: float64(10), int64(45), object(52)
memory usage: 14.1+ MB


In [14]:
def col_names_std(df):
    import pandas as pd
    import numpy as np
    df.columns = [df.columns[i].lower().replace(' ','_') for i in range(len(df.columns))]
    return df

fifa_df=col_names_std(fifa_df)
fifa_df.head()

,name,age,ova,nationality,club,bov,bp,position,player_photo,club_logo,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,gender
ID,,,,,,,,,,,,,,,,,,,,,
2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,...,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,...,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,...,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,...,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,...,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


In [39]:
fifa_df.sort_values('jumping')
display(fifa_df[['name', 'nationality','bp', 'club', 'jumping']].sort_values('jumping', ascending=False))

,name,nationality,bp,club,jumping
ID,,,,,
20801,Cristiano Ronaldo,Portugal,ST,Juventus,95.0
241002,E. Sabbi,United States,CAM,Odense Boldklub,94.0
238227,K. Diatta,Senegal,RM,Club Brugge KV,94.0
239015,E. Dennis,Nigeria,ST,Club Brugge KV,94.0
233141,T. Hasegawa,Japan,LM,Kawasaki Frontale,94.0
...,...,...,...,...,...
184384,D. Stadelmann,Switzerland,RM,FC Lausanne-Sport,NaN
184522,M. Benvenuto,Italy,CAM,Triestina,NaN
188964,Bragança,Portugal,CF,FC Ingolstadt 04,NaN


In [44]:
##We select those FIFA attributes that are relevant for the scoring of goals.

#The categories can be found here:
#https://fifauteam.com/fifa-20-attributes-guide/

#From all the categories, we select those attributes corresponding to Pace, Dribbling, Shooting and Physical.

#We also include the best position (bp) in the model

entries_interest=['name','age','bp','acceleration','sprint_speed','agility','balance','ball_control','composure','dribbling',
                 'reactions','finishing','long_shots', 'penalties','positioning','shot_power','volleys', 'aggression',
                 'jumping','stamina','strength']

In [45]:
fifa_df_model=fifa_df[entries_interest]
fifa_df_model

,name,age,bp,acceleration,sprint_speed,agility,balance,ball_control,composure,dribbling,...,finishing,long_shots,penalties,positioning,shot_power,volleys,aggression,jumping,stamina,strength
ID,,,,,,,,,,,,,,,,,,,,,
2,G. Pasquale,33,LWB,68,74,68.0,68.0,71,NaN,73,...,50,68,50,63.0,74,58.0,72,68.0,69,68
16,Luis García,37,CM,56,50,62.0,72.0,71,79.0,69,...,64,71,75,72.0,75,68.0,71,54.0,64,60
27,J. Cole,33,CAM,48,42,71.0,75.0,78,NaN,79,...,64,69,66,69.0,72,75.0,69,58.0,29,56
36,D. Yorke,36,ST,59,62,NaN,NaN,77,NaN,68,...,70,59,70,72.0,63,NaN,59,NaN,51,66
41,Iniesta,36,CAM,61,56,79.0,75.0,90,89.0,85,...,69,70,71,78.0,67,79.0,58,40.0,58,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259088,A. Medioub,22,CB,52,55,46.0,42.0,56,46.0,44,...,23,28,39,23.0,48,26.0,69,70.0,59,74
259090,W. Rickard,18,CAM,61,63,52.0,68.0,54,46.0,57,...,43,48,50,55.0,51,41.0,51,62.0,53,57
259091,C. Barrett,18,RB,66,61,54.0,68.0,47,43.0,52,...,34,37,49,49.0,28,34.0,58,61.0,53,54
